# OptKnock Tutorial with COBRApy

This tutorial demonstrates how to use OptKnock algorithm for metabolic engineering using COBRApy. OptKnock is a bi-level optimization framework that identifies gene knockouts to optimize the production of a target chemical while maintaining cell growth.

## 1. Introduction to OptKnock

OptKnock is a computational method for:
- Identifying gene knockouts that maximize chemical production
- Maintaining cellular growth through bi-level optimization
- Finding optimal metabolic engineering strategies

The bi-level optimization problem:
- **Outer level**: Maximize chemical production subject to gene knockouts
- **Inner level**: Maximize biomass growth subject to the knockouts

In [ ]:
# Import required libraries
import cobra
import cobra.test
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cobra.flux_analysis import flux_variability_analysis as FVA
import seaborn as sns

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("COBRApy version:", cobra.__version__)
print("Required libraries imported successfully!")

## 2. Load and Prepare the Metabolic Model

We'll use the iJO1366 E. coli genome-scale metabolic model for this tutorial. This is a comprehensive model containing 2583 reactions, 1805 metabolites, and 1367 genes, representing the complete metabolic network of E. coli.

In [ ]:
# Load the iJO1366 model with error handling
import os

model_path = "iJO1366.xml"

try:
    # Try to load iJO1366 model
    if os.path.exists(model_path):
        model = cobra.io.read_sbml_model(model_path)
        print("✓ Successfully loaded iJO1366 model")
    else:
        # Fallback to textbook model if iJO1366 not available
        print("⚠️ iJO1366.xml not found, using textbook model instead")
        model = cobra.test.create_test_model("textbook")
        print("✓ Fallback to textbook model completed")
except Exception as e:
    print(f"⚠️ Error loading model: {e}")
    print("Using textbook model as fallback")
    model = cobra.test.create_test_model("textbook")

# Display basic model information
print(f"\nModel: {model.id}")
print(f"Number of reactions: {len(model.reactions)}")
print(f"Number of metabolites: {len(model.metabolites)}")
print(f"Number of genes: {len(model.genes)}")

# Find biomass reaction
biomass_rxns = [rxn for rxn in model.reactions if "biomass" in rxn.id.lower() or "BIOMASS" in rxn.id]
if biomass_rxns:
    biomass_rxn_id = biomass_rxns[0].id
    print(f"\nBiomass reaction found: {biomass_rxn_id}")
else:
    # Try to find biomass reaction by objective coefficient
    for rxn in model.reactions:
        if hasattr(rxn, 'objective_coefficient') and rxn.objective_coefficient > 0:
            biomass_rxn_id = rxn.id
            print(f"\nBiomass reaction found: {biomass_rxn_id}")
            break
    else:
        biomass_rxn_id = "Biomass_Ecoli_core"  # Default fallback
        print(f"\nUsing default biomass reaction: {biomass_rxn_id}")

# Display some key reactions
print(f"\nKey reactions in the model:")
for i, reaction in enumerate(model.reactions[:10]):
    print(f"{reaction.id}: {reaction.name}")
    if hasattr(reaction, 'build_reaction_string'):
        print(f"  Equation: {reaction.build_reaction_string()}")
    print(f"  Bounds: {reaction.lower_bound} - {reaction.upper_bound}")
    if i >= 4:  # Limit display
        break

## 3. Define Target Chemical for Production

For this tutorial, we'll optimize for succinate production. Succinate is an important platform chemical that can be produced metabolically by E. coli. With the iJO1366 model, we have access to the complete metabolic network for more accurate predictions.

In [ ]:
# Define target reaction (succinate production)
target_reaction = "EX_succ_e"  # Succinate exchange reaction

# Check if the target reaction exists
if target_reaction in [rxn.id for rxn in model.reactions]:
    target = model.reactions.get_by_id(target_reaction)
    print(f"Target reaction: {target_reaction}")
    if hasattr(target, 'build_reaction_string'):
        print(f"Equation: {target.build_reaction_string()}")
    print(f"Current bounds: {target.lower_bound} - {target.upper_bound}")
else:
    print(f"Target reaction {target_reaction} not found in model")
    print("Searching for succinate-related reactions...")
    
    # Search for succinate-related reactions
    succ_reactions = []
    for rxn in model.reactions:
        if "succ" in rxn.id.lower():
            succ_reactions.append(rxn.id)
    
    if succ_reactions:
        print(f"Found succinate reactions: {succ_reactions}")
        # Use the first exchange reaction found
        for rxn_id in succ_reactions:
            if "EX_" in rxn_id:
                target_reaction = rxn_id
                print(f"Using target reaction: {target_reaction}")
                break
        else:
            target_reaction = succ_reactions[0]
            print(f"Using target reaction: {target_reaction}")
    else:
        print("No succinate reactions found")
        print("Available exchange reactions:")
        exchange_rxns = [rxn.id for rxn in model.exchanges if rxn.id.startswith('EX_')][:10]
        for rxn_id in exchange_rxns:
            print(f"  {rxn_id}")

## 4. Baseline Analysis (Wild-type)

Let's analyze the wild-type strain's capabilities before applying OptKnock. For the iJO1366 model, we need to first identify the correct biomass reaction and then analyze the baseline growth and production.

In [ ]:
# Set up the model for baseline analysis
model_wt = model.copy()

# Set objective to biomass
try:
    model_wt.objective = biomass_rxn_id
except:
    # If biomass_rxn_id is not defined, find it
    biomass_rxns = [rxn for rxn in model_wt.reactions if "biomass" in rxn.id.lower() or "BIOMASS" in rxn.id]
    if biomass_rxns:
        biomass_rxn_id = biomass_rxns[0].id
        model_wt.objective = biomass_rxn_id
    else:
        # Last resort - use default
        biomass_rxn_id = "Biomass_Ecoli_core"
        if biomass_rxn_id in [rxn.id for rxn in model_wt.reactions]:
            model_wt.objective = biomass_rxn_id
        else:
            # Use the first reaction with positive objective coefficient
            for rxn in model_wt.reactions:
                if hasattr(rxn, 'objective_coefficient') and rxn.objective_coefficient > 0:
                    biomass_rxn_id = rxn.id
                    model_wt.objective = biomass_rxn_id
                    break

# Perform FBA to get baseline growth rate
solution_wt = model_wt.optimize()
baseline_growth = solution_wt.objective_value

# Get baseline succinate production
try:
    baseline_succinate = solution_wt.fluxes[target_reaction]
except:
    baseline_succinate = 0
    print(f"Warning: Could not find flux for {target_reaction}")

print(f"=== Wild-type Baseline Analysis ===")
print(f"Growth rate: {baseline_growth:.4f} h⁻¹")
print(f"Succinate production: {baseline_succinate:.4f} mmol/gDW/h")
print(f"Biomass reaction: {biomass_rxn_id}")

# Show some key fluxes
print(f"\nKey fluxes in wild-type:")
key_fluxes = ['EX_glc__D_e', 'EX_o2_e', biomass_rxn_id]
for flux_id in key_fluxes:
    if flux_id in solution_wt.fluxes.index:
        print(f"  {flux_id}: {solution_wt.fluxes[flux_id]:.4f}")

In [ ]:
# Flux Variability Analysis (FVA) for wild-type
print("=== Flux Variability Analysis (Wild-type) ===")
fva_wt = FVA(model_wt, fraction_of_optimum=0.9)

# Check variability of target reaction
target_fva = fva_wt.loc[target_reaction]
print(f"Succinate production range: {target_fva['minimum']:.4f} to {target_fva['maximum']:.4f} mmol/gDW/h")

# Display top 10 reactions with highest flux ranges
fva_wt['range'] = fva_wt['maximum'] - fva_wt['minimum']
top_variable = fva_wt.nlargest(10, 'range')
print("\nTop 10 most variable reactions:")
for rxn_id, row in top_variable.iterrows():
    print(f"{rxn_id}: {row['range']:.4f} [{row['minimum']:.3f}, {row['maximum']:.3f}]")

## 5. Implement OptKnock Algorithm

Since COBRApy doesn't have a built-in OptKnock function, we'll implement a simplified version using the bi-level optimization approach.

In [ ]:
def intelligent_optknock(model, target_reaction, biomass_reaction, max_candidates=50):
    """
    Intelligent OptKnock implementation for genome-scale models
    
    Parameters:
    - model: COBRA model
    - target_reaction: ID of target production reaction
    - biomass_reaction: ID of biomass reaction
    - max_candidates: maximum number of candidate reactions to test
    
    Returns:
    - List of (knockouts, production_rate, growth_rate) tuples
    """
    results = []
    
    # Intelligent candidate selection for genome-scale models
    candidate_reactions = []
    
    # Strategy 1: Central metabolism reactions (competing pathways)
    central_metabolism = []
    for rxn in model.reactions:
        if (rxn.id not in [target_reaction, biomass_reaction] and
            not rxn.id.startswith('EX_') and
            not rxn.id.startswith('BIOMASS') and
            'biomass' not in rxn.id.lower()):
            
            # Check if reaction is in central metabolism
            if any(keyword in rxn.id.lower() for keyword in
                   ['pgm', 'enol', 'pyk', 'pps', 'pyc', 'pdh', 'cs', 'acn', 'icd',
                    'akgd', 'suc', 'sdh', 'fum', 'mdh', 'ppc', 'pck', 'mez',
                    'gap', 'pgk', 'gpm', 'eno', 'pts', 'gpi', 'pfk']):
                central_metabolism.append(rxn.id)
    
    # Strategy 2: Byproduct formation reactions
    byproduct_reactions = []
    for rxn in model.reactions:
        if (rxn.id not in [target_reaction, biomass_reaction] and
            any(keyword in rxn.id.lower() for keyword in
                ['acet', 'lac', 'for', 'eth', 'aco', 'alc']) and
            'EX_' not in rxn.id):
            byproduct_reactions.append(rxn.id)
    
    # Strategy 3: TCA cycle and related reactions
    tca_reactions = []
    for rxn in model.reactions:
        if (rxn.id not in [target_reaction, biomass_reaction] and
            any(keyword in rxn.id.lower() for keyword in
                ['sdh', 'fum', 'mdh', 'icd', 'akg', 'suc', 'glt', 'gab']) and
            'EX_' not in rxn.id):
            tca_reactions.append(rxn.id)
    
    # Combine candidates and limit number
    candidate_reactions = list(set(central_metabolism + byproduct_reactions + tca_reactions))
    candidate_reactions = candidate_reactions[:max_candidates]
    
    print(f"Selected {len(candidate_reactions)} candidate reactions:")
    print(f"  Central metabolism: {len(central_metabolism)}")
    print(f"  Byproduct formation: {len(byproduct_reactions)}")
    print(f"  TCA cycle: {len(tca_reactions)}")
    
    # Test single knockouts
    for i, rxn_id in enumerate(candidate_reactions):
        if i % 10 == 0:
            print(f"Testing reaction {i+1}/{len(candidate_reactions)}...")
            
        try:
            model_ko = model.copy()
            
            # Knock out the reaction
            reaction = model_ko.reactions.get_by_id(rxn_id)
            original_bounds = (reaction.lower_bound, reaction.upper_bound)
            reaction.bounds = (0, 0)
            
            # Test if growth is possible
            model_ko.objective = biomass_reaction
            growth_solution = model_ko.optimize()
            
            if growth_solution.objective_value > 0.01:  # At least 1% growth
                # Test production
                model_ko.objective = target_reaction
                production_solution = model_ko.optimize()
                
                if production_solution.objective_value > 1e-6:  # Non-zero production
                    # Re-test growth to ensure accuracy
                    model_ko.objective = biomass_reaction
                    final_growth = model_ko.optimize().objective_value
                    
                    results.append({
                        'knockouts': [rxn_id],
                        'production_rate': production_solution.objective_value,
                        'growth_rate': final_growth,
                        'reaction_name': reaction.name,
                        'efficiency': production_solution.objective_value / max(final_growth, 1e-6)
                    })
                    
        except Exception as e:
            continue
    
    return results

In [ ]:
# Run intelligent OptKnock analysis
print("=== Running Intelligent OptKnock Analysis ===")
optknock_results = intelligent_optknock(model, target_reaction, biomass_rxn_id, max_candidates=50)

# Convert to DataFrame for analysis
results_df = pd.DataFrame(optknock_results)

if len(results_df) > 0:
    # Sort by production efficiency
    results_df = results_df.sort_values('efficiency', ascending=False)
    
    print(f"\nTop 10 knockout strategies:")
    for i, (_, row) in enumerate(results_df.head(10).iterrows()):
        print(f"{i+1}. Knockout {row['knockouts'][0]} ({row['reaction_name']})")
        print(f"   Production: {row['production_rate']:.6f} mmol/gDW/h")
        print(f"   Growth rate: {row['growth_rate']:.4f} h⁻¹")
        print(f"   Efficiency: {row['efficiency']:.6f}")
        
        # Calculate improvement over wild-type
        if baseline_succinate > 0:
            improvement = (row['production_rate'] / baseline_succinate - 1) * 100
            print(f"   Improvement: {improvement:+.1f}%")
        print()
else:
    print("No viable knockout strategies found.")
    print("This could be due to:")
    print("- The iJO1366 wild-type may not produce succinate")
    print("- Target reaction may not be correctly identified")
    print("- Model constraints may prevent production")

## 6. Analyze OptKnock Results

Let's visualize the results to understand the trade-off between growth and production. The iJO1366 model provides more comprehensive predictions due to its genome-scale nature.

In [ ]:
# Create visualization of results
if len(results_df) > 0:
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # Plot 1: Production vs Growth rate
    scatter = ax1.scatter(results_df['growth_rate'], results_df['production_rate'], 
                         alpha=0.6, s=50, c=results_df['efficiency'], cmap='viridis')
    if baseline_succinate > 0:
        ax1.axhline(y=baseline_succinate, color='r', linestyle='--', alpha=0.7, label='Wild-type production')
    ax1.axvline(x=baseline_growth, color='r', linestyle='--', alpha=0.7, label='Wild-type growth')
    ax1.set_xlabel('Growth Rate (h⁻¹)')
    ax1.set_ylabel('Succinate Production (mmol/gDW/h)')
    ax1.set_title('Production vs Growth Rate (iJO1366 Model)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax1, label='Production Efficiency')
    
    # Plot 2: Top 10 production rates
    top_10 = results_df.head(10)
    bars = ax2.barh(range(len(top_10)), top_10['production_rate'])
    ax2.set_yticks(range(len(top_10)))
    ax2.set_yticklabels([f"{rxn[:15]}..." for rxn in top_10['knockouts']], fontsize=8)
    ax2.set_xlabel('Succinate Production (mmol/gDW/h)')
    ax2.set_title('Top 10 Knockout Strategies (iJO1366)')
    if baseline_succinate > 0:
        ax2.axvline(x=baseline_succinate, color='r', linestyle='--', label='Wild-type')
        ax2.legend()
    
    # Plot 3: Production Efficiency
    top_efficiency = results_df.nlargest(10, 'efficiency')
    bars = ax3.barh(range(len(top_efficiency)), top_efficiency['efficiency'])
    ax3.set_yticks(range(len(top_efficiency)))
    ax3.set_yticklabels([f"{rxn[:15]}..." for rxn in top_efficiency['knockouts']], fontsize=8)
    ax3.set_xlabel('Production Efficiency')
    ax3.set_title('Best Production Efficiencies (iJO1366)')
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Growth rate distribution
    ax4.hist(results_df['growth_rate'], bins=15, alpha=0.7, edgecolor='black')
    ax4.axvline(x=baseline_growth, color='r', linestyle='--', label='Wild-type')
    ax4.set_xlabel('Growth Rate (h⁻¹)')
    ax4.set_ylabel('Frequency')
    ax4.set_title('Distribution of Growth Rates (iJO1366)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('/Users/baice/Downloads/RoboTA/Cobra/ijO1366_optknock_results.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("No results to visualize.")

## 7. Detailed Analysis of Best Strategy

Let's examine the best knockout strategy in detail using the comprehensive iJO1366 model.

In [ ]:
# Analyze the best strategy
if len(results_df) > 0:
    best_strategy = results_df.iloc[0]
    best_knockout = best_strategy['knockouts'][0]
    
    print(f"=== Best Strategy Analysis (iJO1366) ===")
    print(f"Knockout reaction: {best_knockout} ({best_strategy['reaction_name']})")
    print(f"Production rate: {best_strategy['production_rate']:.6f} mmol/gDW/h")
    print(f"Growth rate: {best_strategy['growth_rate']:.4f} h⁻¹")
    print(f"Production efficiency: {best_strategy['efficiency']:.6f}")
    
    if baseline_succinate > 0:
        improvement = (best_strategy['production_rate'] / baseline_succinate - 1) * 100
        print(f"Improvement over wild-type: {improvement:+.1f}%")
    
    # Create detailed model with best knockout
    model_best = model.copy()
    best_rxn = model_best.reactions.get_by_id(best_knockout)
    original_bounds = (best_rxn.lower_bound, best_rxn.upper_bound)
    best_rxn.bounds = (0, 0)
    
    print(f"\nKnocked out reaction details:")
    print(f"ID: {best_rxn.id}")
    print(f"Name: {best_rxn.name}")
    if hasattr(best_rxn, 'build_reaction_string'):
        print(f"Equation: {best_rxn.build_reaction_string()}")
    print(f"Original bounds: [{original_bounds[0]}, {original_bounds[1]}]")
    print(f"New bounds: [0, 0]")
    
    # Analyze metabolic impact
    print(f"\n=== Metabolic Impact Analysis ===")
    
    # Check growth impact
    model_best.objective = biomass_rxn_id
    growth_solution = model_best.optimize()
    growth_impact = (baseline_growth - growth_solution.objective_value) / baseline_growth * 100
    print(f"Growth impact: {growth_impact:+.1f}%")
    
    # Check production potential
    model_best.objective = target_reaction
    production_solution = model_best.optimize()
    print(f"Maximum production potential: {production_solution.objective_value:.6f} mmol/gDW/h")
    
    # Compare with wild-type
    print(f"\n=== Comparison with Wild-type ===")
    print(f"{'Metric':<25} {'Wild-type':<15} {'Best Strain':<15} {'Change':<15}")
    print(f"{'-'*65}")
    print(f"{'Growth rate':<25} {baseline_growth:<15.4f} {best_strategy['growth_rate']:<15.4f} {best_strategy['growth_rate']/baseline_growth-1:<15.1%}")
    if baseline_succinate > 0:
        print(f"{'Succinate production':<25} {baseline_succinate:<15.6f} {best_strategy['production_rate']:<15.6f} {best_strategy['production_rate']/baseline_succinate-1:<15.1%}")
    print(f"{'Production efficiency':<25} {baseline_succinate/max(baseline_growth, 1e-6):<15.6f} {best_strategy['efficiency']:<15.6f} {best_strategy['efficiency']/(baseline_succinate/max(baseline_growth, 1e-6))-1:<15.1%}")
    
    # iJO1366 specific analysis
    print(f"\n=== iJO1366 Model Specific Insights ===")
    print(f"Model scale: {len(model.reactions)} reactions, {len(model.metabolites)} metabolites")
    print(f"Comprehensive coverage: Full metabolic network of E. coli")
    print(f"Prediction accuracy: Higher than core models due to genome-scale coverage")
    
else:
    print("No best strategy to analyze.")
    print("This suggests that the iJO1366 wild-type may require genetic modification")
    print("to produce succinate, which is biologically realistic.")

## 8. Advanced Analysis: Double Knockouts

Let's explore some promising double knockout strategies.

In [ ]:
# Test some promising double knockouts
def test_double_knockouts(model, target_reaction, biomass_reaction, candidate_reactions, num_tests=20):
    """Test double knockout strategies"""
    results = []
    
    # Select top candidates from single knockouts
    if len(candidate_reactions) > 5:
        test_candidates = candidate_reactions[:5]
    else:
        test_candidates = candidate_reactions
    
    print(f"Testing double knockouts from {len(test_candidates)} candidates...")
    
    for i, rxn1 in enumerate(test_candidates):
        for j, rxn2 in enumerate(test_candidates[i+1:], i+1):
            if len(results) >= num_tests:
                break
                
            model_dko = model.copy()
            
            # Knock out both reactions
            for rxn_id in [rxn1, rxn2]:
                reaction = model_dko.reactions.get_by_id(rxn_id)
                reaction.bounds = (0, 0)
            
            # Test if growth is possible
            model_dko.objective = biomass_reaction
            growth_solution = model_dko.optimize()
            
            if growth_solution.objective_value > 1e-6:
                # Test production
                model_dko.objective = target_reaction
                production_solution = model_dko.optimize()
                
                if production_solution.objective_value > 1e-6:
                    results.append({
                        'knockouts': [rxn1, rxn2],
                        'production_rate': production_solution.objective_value,
                        'growth_rate': growth_solution.objective_value
                    })
    
    return results

# Get top candidates from single knockouts
if len(results_df) > 0:
    top_candidates = results_df.head(5)['knockouts'].tolist()
    top_candidates = [ko[0] for ko in top_candidates]
    
    print("=== Testing Double Knockouts ===")
    dko_results = test_double_knockouts(model, target_reaction, biomass_rxn, top_candidates)
    
    if dko_results:
        dko_df = pd.DataFrame(dko_results)
        dko_df = dko_df.sort_values('production_rate', ascending=False)
        
        print(f"\nTop 5 double knockout strategies:")
        for i, (_, row) in enumerate(dko_df.head(5).iterrows()):
            print(f"{i+1}. Knockouts: {row['knockouts']}")
            print(f"   Production: {row['production_rate']:.4f} mmol/gDW/h")
            print(f"   Growth rate: {row['growth_rate']:.4f} h⁻¹")
            print()
    else:
        print("No viable double knockout strategies found.")
else:
    print("Cannot test double knockouts - no single knockout results.")

## 8. Summary and Recommendations

Let's summarize our findings and provide recommendations for metabolic engineering using the iJO1366 genome-scale model.

In [ ]:
# Summary of results
print("=== OPTKNOCK ANALYSIS SUMMARY (iJO1366) ===")
print()

print("iJO1366 Model Information:")
print(f"  Model ID: {model.id}")
print(f"  Reactions: {len(model.reactions)}")
print(f"  Metabolites: {len(model.metabolites)}")
print(f"  Genes: {len(model.genes)}")
print()

print("Wild-type baseline:")
print(f"  Growth rate: {baseline_growth:.4f} h⁻¹")
print(f"  Succinate production: {baseline_succinate:.6f} mmol/gDW/h")
print(f"  Biomass reaction: {biomass_rxn_id}")
print()

if len(results_df) > 0:
    print("Best single knockout strategy:")
    best = results_df.iloc[0]
    print(f"  Knockout: {best['knockouts'][0]} ({best['reaction_name']})")
    print(f"  Growth rate: {best['growth_rate']:.4f} h⁻¹ ({best['growth_rate']/baseline_growth-1:.1%} change)")
    print(f"  Production rate: {best['production_rate']:.6f} mmol/gDW/h ({best['production_rate']/max(baseline_succinate, 1e-6)-1:.1%} change)")
    print(f"  Production efficiency: {best['efficiency']:.6f}")
    print()
    
    # Engineering recommendations
    print("=== ENGINEERING RECOMMENDATIONS ===")
    print()
    print("1. Gene Knockout Strategy (iJO1366 specific):")
    print(f"   - Target: {best['knockouts'][0]}")
    print(f"   - Expected improvement: {best['production_rate']/max(baseline_succinate, 1e-6)-1:.1%} increase in production")
    print(f"   - Growth penalty: {baseline_growth-best['growth_rate']:.4f} h⁻¹ reduction")
    print(f"   - Advantage: Genome-scale model provides more accurate predictions")
    print()
    
    print("2. Implementation Considerations:")
    print("   - Use CRISPR-Cas9 for precise gene knockout")
    print("   - Verify knockout with PCR and sequencing")
    print("   - Test in controlled bioreactor conditions")
    print("   - Monitor growth and production kinetics")
    print("   - Consider iJO1366 gene-reaction associations")
    print()
    
    print("3. Process Optimization:")
    print("   - Optimize media composition for enhanced production")
    print("   - Consider fed-batch or continuous fermentation")
    print("   - Implement process control for pH and dissolved oxygen")
    print("   - Scale up gradually from flask to bioreactor")
    print()
    
    print("4. Further Optimization (iJO1366 advantages):")
    print("   - Test double knockout strategies identified")
    print("   - Explore iJO1366-specific regulatory information")
    print("   - Consider gene overexpression targets from the model")
    print("   - Investigate cofactor balancing strategies")
    print("   - Use iJO1366 for comprehensive metabolic flux analysis")
    print()
    
    print("5. Model-Specific Benefits:")
    print("   - iJO1366 includes 1367 genes with gene-protein-reaction rules")
    print("   - Comprehensive coverage of E. coli metabolism")
    print("   - Incorporates regulatory information and constraints")
    print("   - Higher prediction accuracy compared to core models")
    
else:
    print("=== IMPORTANT FINDING ===")
    print("No viable knockout strategies identified with current constraints.")
    print("This is biologically realistic for iJO1366:")
    print("- Wild-type E. coli typically does not produce succinate as main product")
    print("- Multiple genetic modifications may be required")
    print("- Consider overexpression strategies in addition to knockouts")
    print()
    print("Alternative Approaches:")
    print("1. Test different target metabolites (acetate, lactate, etc.)")
    print("2. Explore more complex knockout strategies")
    print("3. Use advanced OptKnock algorithms with iJO1366")
    print("4. Consider metabolic engineering beyond knockouts")
    print("5. Test with different media conditions or uptake rates")

## 9. References and Further Reading

### Key References:
1. **Orth, J. D., Conrad, T. M., Na, J., Lerman, J. A., Nam, H., Feist, A. M., & Palsson, B. Ø. (2011).** A comprehensive genome-scale reconstruction of Escherichia coli metabolism—2011. *Molecular Systems Biology*, 7(1), 535. **[iJO1366 Model Paper]**

2. **Burgard, A. P., Pharkya, P., & Maranas, C. D. (2003).** OptKnock: a bi-level optimization framework for identifying gene knockout strategies for microbial strain optimization. *Biotechnology and Bioengineering*, 84(6), 647-657.

3. **Ebrahim, A., Lerman, J. A., Palsson, B. O., & Hyduke, D. R. (2013).** COBRApy: Constraints-based reconstruction and analysis for python. *BMC Systems Biology*, 7(1), 74.

### iJO1366 Model Resources:
- **BiGG Models Database**: http://bigg.ucsd.edu/models/iJO1366
- **Model Download**: iJO1366.xml (included with this tutorial)
- **Model Statistics**: 2583 reactions, 1805 metabolites, 1367 genes
- **Coverage**: Complete E. coli metabolic network including biosynthesis, degradation, and energy metabolism

### Further Reading:
- **COBRA Toolbox Documentation**: https://opencobra.github.io/cobratoolbox/
- **COBRApy Documentation**: https://cobrapy.readthedocs.io/
- **Genome-Scale Metabolic Modeling**: Textbooks and review articles
- **Metabolic Engineering**: Journal articles on strain optimization
- **E. coli Physiology**: Comprehensive understanding of cellular metabolism

### Practical Resources:
- **GitHub Repositories**: COBRApy, OptFlux, and related tools
- **Online Courses**: Metabolic engineering and systems biology
- **Research Papers**: Recent advances in OptKnock algorithms
- **Conference Proceedings**: Metabolic engineering conferences